# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [237]:
!nvidia-smi

Fri Jul 15 01:10:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    29W /  70W |   1128MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. BlazingSQL
  1. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [238]:
!pip install pynvml 
import pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [239]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

# RAPIDS is now installed on Colab.  You can copy your code into the cells below.  Enjoy!

In [ ]:
import cudf
import cupy as cp
import cuml
from cuml.svm import SVR
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv(r'data.csv')

In [ ]:
df.drop('Pulse_Duration_(fs)', axis= 1, inplace= True)
df.drop('Spot_Size_(FWHM um)', axis= 1, inplace= True)

In [ ]:
df.head()

In [ ]:
df2 = df.copy()

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(df2[['Intensity_(W_cm2)', 'Target_Thickness (um)']])

In [ ]:
df2[['Intensity_(W_cm2)', 'Target_Thickness (um)']] = scaler.transform(df2[['Intensity_(W_cm2)', 'Target_Thickness (um)']])

In [ ]:
df2.head()

In [ ]:
X_var = list(df2.columns)
X_var.remove('Max_Proton_Energy_(MeV)')
y_var = ['Max_Proton_Energy_(MeV)']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df2[X_var], df2[y_var], test_size=0.33, random_state=42)

In [ ]:
X_train_cu = cudf.DataFrame.from_pandas(X_train)
X_test_cu = cudf.DataFrame.from_pandas(X_test)
y_train_cu = cudf.DataFrame.from_pandas(y_train)
y_test_cu = cudf.DataFrame.from_pandas(y_test)

In [ ]:
start_time = time.time()

In [ ]:
model = SVR(kernel='rgf', gamma='auto', C=51.2, epsilon=0.088, coef0=.082, cache_size= 10000, nochange_steps= 5000)

In [ ]:
model.fit(X_train_cu, y_train_cu)

In [ ]:
model.predict(X_test)

In [ ]:
end_time= time.time()

In [ ]:
total_time = end_time - start_time

# Summary

In [ ]:
print(f'Size of training set| {len(X_train)}\nSize of testing set | {len(X_test)}')

In [ ]:
y_predict = cp.asnumpy(model.predict(X_test))
mean_squared_error(y_test, y_predict)

In [ ]:
print(f'Wall Clock Time\n\nHours  | {total_time // 60**2}\nMinutes| {total_time // 60}\nSeconds| {np.abs(total_time - 60 * (total_time // 60 ))}')

In [ ]:
r2_score(y_test, model.predict(y_test))

In [ ]:
y_predict